## Preliminaries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os

Set paths

In [ ]:
abundance_path = '../data/MAG_coverage.tsv'
raw_rna_dir = '../data/transcriptomes/raw/'
rna_dir = '../data/transcriptomes/normalised/'
test_dir = '../data/models/memote_tests/'
abundance_cutoff = 0.01
sample_names = ['BH1', 'BH2', 'BH3', 'SAH1', 'SAH2', 'SAH3', 'AH1', 'AH2', 'AH3']

Load MAG abundances

In [ ]:
abundance_table = pd.read_table(abundance_path, index_col='Bin Id')
abundance_table = abundance_table.iloc[:, abundance_table.columns.str.contains('% community', regex=False)]
abundance_table.columns = abundance_table.columns.str.replace('.sorted: % community', '')
abundance_table.index = abundance_table.index.str.replace('.fasta', '')
abundance_table.columns = sample_names
MAG_names = abundance_table.index

Load RNA counts and build global RNA tables

In [ ]:
df_list = []
df_raw_list = []
key_list = []
for m in MAG_names:
    counts = pd.read_table(rna_path + m + '.csv_rlog', sep=',', index_col='Unnamed: 0')
    counts.index = counts.index.str.replace('.faa_', '__')
    df_list.append(counts)
    raw_counts = pd.read_table(raw_rna_path + m + '.csv', sep=',', index_col='Unnamed: 0')
    raw_counts.index = raw_counts.index.str.replace('.faa_', '__')
    df_raw_list.append(raw_counts)
    key_list.append(m)
rna_table = pd.concat(df_list, axis=0, keys=key_list)
raw_rna_table = pd.concat(df_raw_list, axis=0, keys=key_list)
rna_table.columns = sample_names
raw_rna_table.columns = sample_names

Sum RNA abundances

In [ ]:
raw_rna_sum_table = pd.DataFrame(index=MAG_names, columns=sample_names)
rna_sum_table = pd.DataFrame(index=MAG_names, columns=sample_names)
for s in sample_names:
    count_sums = raw_rna_table[s].sum(axis=0, level=0)
    raw_rna_sum_table.loc[:, s] = np.log10(count_sums) # log10 for visualisation purposes
    count_sums = rna_table[s].sum(axis=0, level=0)
    rna_sum_table.loc[:, s] = np.log(count_sums) # ln for filtering purposes
rna_sum_table = rna_sum_table.divide(rna_sum_table.sum(axis=0))

Combine nucleic acid abundances

In [ ]:
multiomic_abundance_table = np.multiply(abundance_table, rna_sum_table)
multiomic_abundance_table = multiomic_abundance_table.divide(multiomic_abundance_table.sum(axis=0))

Get MAG names for abundant species

In [ ]:
MAG_names = []
for s in sample_names:
    MAG_names.extend(list(abundance_table.index[multiomic_abundance_table[s] >= abundance_cutoff]))
MAG_names = np.unique(MAG_names)

Subset DNA and RNA tables

In [ ]:
abundance_table = abundance_table.loc[MAG_names, :]
rna_table = rna_table.loc[MAG_names, :]
full_raw_rna_sum_table = raw_rna_sum_table
raw_rna_sum_table = raw_rna_sum_table.loc[MAG_names, :]

Rename MAGs with taxonomic associations

In [ ]:
taxa = ['Aneurinibacillaceae sp.', 'Firmicutes sp. 2', 'Acetivibrionaceae sp.', \
    'Bacteroidales sp. 1', 'Limnochordia sp. 2', 'C. proteolyticus', \
    'Bacteroidales sp. 2', 'Methanothermobacter sp.', 'Firmicutes sp. 1', \
    'Firmicutes sp. 3', 'Lutisporaceae sp.', 'Methanobacterium sp.', \
    'Competibacteraceae sp.', 'Firmicutes sp. 6', 'Acetomicrobium sp. 2', \
    'Acetomicrobium flavidum', 'Methanosarcina sp.', 'Tepidiphilus sp.', \
    'Defluviitoga tunisiensis', 'Firmicutes sp. 7', 'Oscillospirales sp. 2', \
    'Methanoculleus sp.', 'Firmicutes sp. 5', 'Thermodesulfovibrio sp.', 'Acetomicrobium sp. 1', \
    'Oscillospirales sp. 1', 'Firmicutes sp. 4', 'Limnochordia sp. 1']
MAG_dict = pd.Series(taxa, index=['bin_10_operams', 'bin_10_unicycler', 'bin_13_unicycler',
       'bin_15_metaspades', 'bin_15_operams', 'bin_16_metaspades',
       'bin_19_unicycler', 'bin_23_metaspades', 'bin_24_unicycler',
       'bin_25_unicycler', 'bin_26_metaspades', 'bin_26_operams',
       'bin_2_unicycler', 'bin_31_metaspades', 'bin_35_metaspades',
       'bin_36_metaspades', 'bin_37_operams', 'bin_43_metaspades',
       'bin_4_operams', 'bin_50_metaspades', 'bin_51_operams',
       'bin_57_operams', 'bin_58_operams', 'bin_5_unicycler', 'bin_64_operams',
       'bin_7_metaspades', 'bin_7_unicycler', 'bin_9_metaspades'])
abundance_table.index = list(MAG_dict[abundance_table.index])
raw_rna_sum_table.index = list(MAG_dict[raw_rna_sum_table.index])
sort_idx = abundance_table.mean(axis=1).argsort()
abundance_table = abundance_table.iloc[sort_idx[::-1], :]
raw_rna_sum_table = raw_rna_sum_table.iloc[sort_idx[::-1], :]

## Plots

MAG abundances

In [ ]:
plt.clf()
plt.cla()
sns.set_context("paper")
fig, axes = plt.subplots(2, 3, figsize=(6, 7), sharex='col', gridspec_kw={'height_ratios': [1, 10], \
    'width_ratios': [4.5, 3, 0.2], 'hspace': 0.02})
axes[0,0].axis('off')
axes[1,0].axis('off')
axes[0,2].axis('off')
axes[0,1].bar(range(9)+np.array([0.5]*9), abundance_table.sum(), align='center', color=[0.015, 0.282, 0.121])
axes[0,1].set_xlabel('')
axes[0,1].set_ylabel('Total')
axes[0,1].tick_params(axis='x', bottom=False)
axes[0,1].set_yticks([0, 90])
axes[0,1].set_yticklabels(['0', '90%'])
sns.heatmap(abundance_table, vmin=0, vmax=35, cmap='Greens', square=True, ax=axes[1,1], cbar_ax=axes[1,2], \
    cbar_kws={"orientation": "vertical", "shrink": .4, 'format': '%.0f%%', 'ticks': [0, 35], 'label': 'Coverage'}) # cmap=cmap, annot=True,
axes[1,1].set_ylabel('')
axes[1,2].set_aspect(10)
# make frame visible
for _, spine in axes[1,1].spines.items():
    spine.set_visible(True)

RNA counts

In [ ]:
plt.clf()
plt.cla()
sns.set_context("paper")
fig, axes = plt.subplots(2, 3, figsize=(6, 7), sharex='col', gridspec_kw={'height_ratios': [1, 10], \
    'width_ratios': [4.5, 3, 0.2], 'hspace': 0.02})
axes[0,0].axis('off')
axes[1,0].axis('off')
axes[0,2].axis('off')
axes[0,1].bar(range(9)+np.array([0.5]*9), 100*raw_rna_sum_table.sum().div(raw_rna_sum_table.sum()), align='center', color=[0.044, 0.15, 0.28])
axes[0,1].set_xlabel('')
axes[0,1].set_ylabel('Total')
axes[0,1].tick_params(axis='x', bottom=False)
axes[0,1].set_yticks([0, 90])
axes[0,1].set_yticklabels(['0', '90%'])
sns.heatmap(raw_rna_sum_table, cmap='Blues', square=True, ax=axes[1,1], cbar_ax=axes[1,2], \
    cbar_kws={"orientation": "vertical", "shrink": .4, 'ticks': [1, 3, 5, 7], 'label': 'Count sum (log10)'}) # cmap=cmap, annot=True,
axes[1,1].set_ylabel('')
axes[1,2].set_aspect(10)
# make frame visible
for _, spine in axes[1,1].spines.items():
    spine.set_visible(True)

MEMOTE test results

In [ ]:
result_df = pd.DataFrame(columns=['model', 'test', 'score'])
json_files = [f for f in os.listdir(test_dir) if ".json" in f]
for f in json_files:
    print(f + ': ')
    with open(test_dir + f) as json_file:
        data = json.load(json_file)
    for t in data['tests'].keys():
        if isinstance(data['tests'][t]['data'], list):
            result_df = result_df.append(pd.DataFrame({'model': [f.strip('.json')], \
                'Test': [t], 
                'Score': [data['tests'][t]['metric']], 
                'Data': [len(data['tests'][t]['data'])],
                'Title': [data['tests'][t]['title']]}), ignore_index=True)
        else:
            result_df = result_df.append(pd.DataFrame({'model': [f.strip('.json')], \
                'Test': [t], 
                'Score': [data['tests'][t]['metric']], 
                'Data': [0],
                'Title': [data['tests'][t]['title']]}), ignore_index=True)
idx = [type(result_df.loc[i, 'Score']) is float for i in range(len(result_df['Score']))]
result_df = result_df.loc[idx, :]
result_df['Score'] = pd.to_numeric(result_df['Score'])
result_df['x'] = ['This study']*len(result_df['Score'])

# exclude non-relevant tests
excluded_tests = ['test_absolute_extreme_coefficient_ratio', \
    'test_biomass_presence', 
    'test_biomass_specific_sbo_presence',
    'test_compartments_presence',
    'test_degrees_of_freedom',
    'test_demand_specific_sbo_presence',
    'test_exchange_specific_sbo_presence',
    'test_fbc_presence',
    'test_find_constrained_pure_metabolic_reactions',
    'test_find_constrained_transport_reactions',
    'test_find_duplicate_metabolites_in_compartments',
    'test_find_duplicate_reactions',
    'test_find_reactions_with_identical_genes',
    'test_find_reactions_with_partially_identical_annotations',
    'test_find_reversible_oxygen_reactions',
    'test_find_unique_metabolites',
    'test_gene_product_annotation_presence',
    'test_gene_protein_reaction_rule_presence',
    'test_gene_sbo_presence',
    'test_gene_specific_sbo_presence',
    'test_matrix_rank',
    'test_metabolic_reaction_specific_sbo_presence',
    'test_metabolite_annotation_presence',
    'test_metabolite_id_namespace_consistency',
    'test_metabolite_sbo_presence',
    'test_metabolite_specific_sbo_presence',
    'test_model_id_presence',
    'test_ngam_presence',
    'test_number_independent_conservation_relations',
    'test_protein_complex_presence',
    'test_reaction_annotation_presence',
    'test_reaction_id_namespace_consistency',
    'test_reaction_sbo_presence',
    'test_sbml_level',
    'test_sink_specific_sbo_presence',
    'test_transport_reaction_gpr_presence',
    'test_transport_reaction_specific_sbo_presence']
result_df = result_df.loc[~result_df['Test'].isin(excluded_tests), :]
# ensure relevant information
result_df.loc[result_df['Test']=='test_find_medium_metabolites', 'Score'] = \
    result_df.loc[result_df['Test']=='test_find_medium_metabolites', 'Data']
result_df.loc[result_df['Test']=='test_metabolites_presence', 'Score'] = \
    result_df.loc[result_df['Test']=='test_metabolites_presence', 'Data']
result_df.loc[result_df['Test']=='test_reactions_presence', 'Score'] = \
    result_df.loc[result_df['Test']=='test_reactions_presence', 'Data']
result_df.loc[result_df['Test']=='test_genes_presence', 'Score'] = \
    result_df.loc[result_df['Test']=='test_genes_presence', 'Data']

plt.clf()
plt.cla()
sns.set_context("paper")
p = sns.catplot(data=result_df, x='x', y='Score', col='Title', kind='violin', \
    col_wrap=5, height=2, aspect=1.5, sharey=False)
for i in range(len(p.axes)):
    ax = p.axes[i]
    ax.set_xlabel('')
    ax.set(xticklabels=[])
    ax.tick_params(bottom=False)
p.set_titles("{col_name}")